
Merge BIT image similarity and USE text similarity

Author: Christopher Pietsch

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import umap
from sklearn.preprocessing import Normalizer

In [10]:
data = pd.read_csv("export1305.csv").set_index('id')
data

,idlong,sammlung,idnr,titel,sachbegriff,actors,ort,datum,material,abmessung,beschreibung,datum_clean
id,,,,,,,,,,,,
968378,\tDE-MUS-815114/968378,ang,A I 232,Schafe im Erlenhain,Bild,"Heinrich von Zügel (22.10.1850 - 30.1.1941, Ma...",NaN,1875,Öl auf Leinwand,"Höhe x Breite: 45,5 x 72,5 cm","»Ich hatte das große Glück, niemals an einen B...",1875
959351,\tDE-MUS-815114/959351,ang,A III 730 (01),Einzug der Deutsch-Englischen Legion in Hannov...,Bild & Zyklus & Studie,"Ernst Hildebrand (8.3.1833 - 17.11.1924, Maler)",NaN,um 1900,Öl auf Leinwand,"Höhe x Breite: 65 x 39,5 cm;Rahmenmaß: 83 x 58...",Das mit Großbritannien seit 1714 durch Persona...,1900
962134,\tDE-MUS-815114/962134,ang,A I 1112,Campagna-Landschaft,Bild,"Arnold Böcklin (16.10.1827 - 16.1.1901, Maler)",NaN,um 1859,Öl auf Leinwand,Höhe x Breite: 88 x 105 cm;Rahmenmaß: 119 x 13...,Von 1850 bis 1857 lebte und arbeitete Arnold B...,1859
961042,\tDE-MUS-815114/961042,ang,A I 879,In der Laube,Bild,"Fritz Burger (16.7.1867 - 11.4.1927, Maler)",NaN,vor 1905,Öl auf Leinwand,"Höhe x Breite: 116 x 139,5 cm;Rahmenmaß: 140,5...",Fritz Burger schuf mit diesem Gemälde ein reiz...,1905
963307,\tDE-MUS-815114/963307,ang,A I 268,Der Verbrecher aus verlorener Ehre,Bild,Rudolf Friedrich August Henneberg (13.9.1825 -...,NaN,1860,Öl auf Leinwand,"Höhe x Breite: 197,5 x 135 cm","Allgemein wird vorausgesetzt, daß die authenti...",1860
...,...,...,...,...,...,...,...,...,...,...,...,...
567563,DE-MUS-018719/567563,mek,A (7 L 100) 11/1908,Tabakkasten zur ostfriesischen Winterküche,Tabakkasten zur ostfriesischen Winterküche,"James Simon (17.9.1851 - 23.5.1932, Sammler);C...","Gebrauchsort: Ostfriesland, Hannover",um 1800,"Holz, Messingknopf",Länge x Breite x Höhe: 19 x 15 x 14 cm,"Oval, auf dem Deckel Sechsstern und Messingkno...",1800
1486490,DE-MUS-018719/1486490,mek,D (44 B 1) 230/1966,Stempel der Kupferschmiedegesellen in Rostock,Stempel der Kupferschmiedegesellen in Rostock,NaN,"Gebrauchsort: Rostock, Mecklenburg",1851,"Messing, graviert; Holzgriff, gedrechselt","Höhe: 7,2 cm","Gedrechselter Holzgriff. Runde Messingplatte, ...",1851
1532768,DE-MUS-018719/1532768,mek,"D (62 C 25) 784/1989,b",Töchteralbum. 37. Band,Töchteralbum. 37. Band,Carl Flemming AG (Verleger);Thekla von Gumpert...,Herstellungsort: Glogau,1891,"Ganzleinen mit Deckel- und Rückenprägung, gekl...","Höhe x Breite x Tiefe: 23 x 16 x 3,5 cm",Hochformat. Töchter-Album. Unterhaltungen im h...,1891


In [65]:
df_images = pd.read_csv("pca-bit.csv", index_col=0).set_index('id')
df_images.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_images)
df_images

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1486397,-0.127307,0.239014,-0.002500,-0.000500,0.245514,0.060604,-0.085305,0.067104,0.078805,0.095506,-0.039402,0.050803,-0.027002,0.099006,-0.090505,0.082205,0.161109,0.113007,0.118707,0.015101,-0.002500,0.177110,0.017501,-0.040402,-0.182111,-0.072604,-0.071704,-0.046203,0.255115,-0.043503,-0.093105,-0.028902,-0.123507,0.274416,-0.018401,0.154309,-0.042902,-0.040102,-0.028202,0.066304,...,-0.0010,0.003200,-0.0020,-0.009101,-0.010001,0.001800,0.016001,-0.004900,0.012901,0.021501,0.0021,-0.020201,-0.0084,-0.020701,0.019101,-0.032102,-0.025001,0.0018,-0.0007,-0.0026,0.0049,0.008701,0.0079,-0.0079,-0.0072,0.028402,-0.015301,-0.025501,0.014701,0.005000,0.029102,-0.025501,0.011001,-0.016301,-0.023701,-0.013201,-0.0085,0.0019,-0.008200,0.018401
1570538,0.008800,0.522309,-0.178303,-0.257405,0.014800,0.013400,0.074601,-0.113302,-0.185403,0.095002,-0.086502,0.021900,0.099002,0.176003,0.117302,0.022300,0.155503,0.065001,0.110502,-0.004900,-0.073001,0.039001,-0.097202,-0.007400,-0.001100,-0.187003,-0.109902,0.001700,-0.008000,-0.014400,0.044701,-0.053501,-0.037101,-0.031201,-0.076301,-0.075101,-0.092202,-0.044801,0.022100,0.006100,...,0.0044,0.000500,-0.0012,0.000600,0.021400,0.006600,0.001300,-0.024900,-0.002000,-0.010900,0.0082,0.002700,0.0030,-0.010600,-0.004700,-0.006500,-0.007100,0.0222,0.0127,0.0208,0.0204,-0.005800,-0.0130,-0.0242,-0.0056,-0.029701,-0.007800,-0.002200,-0.012300,-0.005500,0.002900,-0.004600,-0.004800,-0.025600,-0.026600,-0.000800,0.0153,-0.0071,-0.003800,-0.002300
525580,-0.032400,0.540697,0.006800,-0.074000,-0.037900,-0.331498,-0.062200,0.106499,0.063600,-0.010200,0.101899,0.465597,-0.038900,-0.188499,-0.099599,-0.096399,-0.091599,-0.042700,-0.105599,0.041700,-0.042600,-0.036500,0.112199,0.168199,-0.015200,-0.025000,-0.060900,-0.027600,-0.046900,-0.041900,-0.077700,0.027000,-0.089599,-0.086699,-0.084399,-0.022500,-0.013400,0.030800,-0.026000,-0.021300,...,0.0139,0.001800,-0.0014,-0.000300,0.022300,0.002100,-0.005500,0.005500,0.010100,0.009100,-0.0088,0.000700,-0.0127,0.002300,-0.003300,-0.007000,-0.022700,0.0115,-0.0122,-0.0002,-0.0006,0.003300,0.0098,0.0077,0.0038,-0.000600,-0.002000,-0.003200,-0.000400,-0.000900,0.008300,-0.007200,-0.014700,0.001800,0.013800,0.010300,-0.0095,-0.0075,0.001100,-0.000200
1442848,0.154101,0.209401,-0.247101,0.375702,-0.098400,-0.204701,-0.076500,-0.037100,-0.173701,-0.021100,-0.089500,-0.066100,-0.026800,0.044400,0.122201,-0.026500,-0.131201,-0.194001,-0.156001,-0.166701,-0.053900,0.150501,-0.124501,-0.109001,-0.047600,0.080900,-0.039900,0.025600,-0.123401,0.164201,-0.006300,-0.069500,0.032900,-0.033600,-0.105300,0.025200,0.062900,0.065000,0.004800,-0.140801,...,-0.0320,-0.025800,0.0107,0.008700,-0.007100,0.001300,-0.001100,0.006600,0.025000,-0.005400,-0.0075,-0.023200,0.0068,0.028000,-0.004300,-0.010400,0.006500,-0.0073,0.0343,0.0178,0.0137,-0.021000,0.0110,0.0037,0.0054,0.006700,0.007700,-0.002600,-0.003000,0.033700,0.010500,0.010800,0.001600,0.003200,-0.010200,0.000000,0.0088,0.0085,0.003300,0.008100
962622,0.425799,-0.177000,0.329099,0.022600,0.005600,-0.046900,0.047100,-0.063600,0.043100,0.052600,0.029800,-0.012800,-0.237499,-0.159900,0.130200,0.014800,-0.036100,0.068500,0.152400,-0.024000,-0.004000,-0.074600,-0.006800,-0.101300,-0.032800,-0.068600,-0.073000,0.057100,-0.039000,0.057000,-0.009100,0.075100,-0.020400,-0.029100,0.110900,-0.020900,-0.056200,0.168200,-0.018100,-0.050500,...,0.0075,0.016500,-0.0083,-0.010800,0.012000,-0.014800,0.017300,0.007600,-0.003700,-0.016100,-0.0239,0.008600,-0.0102,-0.007800,-0.007000,-0.000700,0.020600,-0.0115,0.0022,-0.0207,-0.0003,-0.007100,0.0107,0.0025,0.0125,0.000400,-0.023400,0.000700,

In [64]:
df_title = pd.read_csv("use.csv", index_col=0).set_index('id')
df_title.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_title)
df_title

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
968378,-0.004004,0.008708,0.068764,-0.024423,0.065361,-0.023322,-0.016716,-0.077572,-0.079674,0.028827,0.031229,0.065461,0.025123,0.044141,-0.020919,-0.000601,-0.000901,0.024823,-0.047744,0.033231,-0.023021,0.015314,-0.034532,0.024423,0.032830,-0.018017,-0.031729,-0.023422,-0.075870,-0.001101,-0.018117,-0.026825,-0.039437,-0.058855,0.081476,0.038536,-0.088182,-0.027425,0.033231,0.011210,...,-0.000200,0.037935,0.080475,0.077172,-0.000300,-0.090184,0.064059,0.004504,-0.013112,-0.055952,-0.046243,-0.014213,0.023422,0.021420,0.023622,0.074269,0.033932,0.023222,0.027626,0.016615,-0.011711,0.003703,0.034532,0.042239,0.033131,0.059755,-0.077272,-0.079073,0.015715,0.054651,0.025324,0.016015,-0.032630,0.016916,-0.050347,0.068163,-0.018417,-0.082677,0.043140,0.017016
959351,0.005205,0.004904,0.000100,-0.053147,0.079470,-0.074066,0.021419,-0.064557,0.100189,-0.009809,0.012011,-0.056650,0.007607,-0.013612,-0.003803,-0.010910,0.020718,0.009709,0.024722,-0.011010,-0.057451,0.049244,-0.007507,0.001201,-0.016915,-0.033430,-0.001501,0.036733,-0.034030,0.011911,-0.001601,-0.055249,0.005805,-0.005405,0.122609,-0.001501,-0.019117,0.003503,-0.092582,0.065658,...,0.015213,-0.075467,-0.047943,0.054448,-0.017215,-0.020018,0.057251,-0.009308,0.017516,-0.002302,-0.048943,-0.037133,-0.047842,-0.038134,0.033330,0.098688,-0.050645,0.000601,-0.062155,0.011610,-0.030727,0.007306,0.010609,-0.034330,-0.048043,-0.019317,0.033830,-0.082373,0.001101,-0.048443,-0.043839,-0.022220,-0.035031,-0.042438,0.046241,0.081472,0.035932,0.002202,-0.068861,0.022120
962134,-0.021520,-0.039136,-0.040938,-0.034932,0.013813,0.114306,0.068664,0.029227,-0.089884,0.003904,0.012011,0.040938,-0.039036,0.089383,-0.036734,0.014513,0.039136,-0.076071,0.020119,0.036034,-0.013012,0.042139,0.014614,0.054150,-0.045642,-0.047144,-0.044141,-0.049346,-0.005405,-0.016215,0.032330,-0.014814,0.011811,-0.046743,-0.000701,0.079774,-0.037235,0.046944,0.065161,-0.077172,...,0.008108,0.003003,-0.025724,0.071767,-0.011811,-0.041238,0.057453,0.076371,-0.013412,0.028126,0.057153,0.024122,0.009909,0.021120,-0.037135,-0.005505,0.027225,0.003703,-0.011511,0.037034,-0.011010,0.025524,0.062358,-0.060156,0.047044,0.047945,-0.010910,-0.072868,0.057754,0.001601,-0.011811,0.065261,0.015314,0.078673,-0.017616,0.014914,-0.082577,0.023021,-0.038836,0.004404
961042,-0.013612,-0.049943,-0.058250,-0.045539,0.049443,0.009808,-0.067358,-0.037432,-0.056849,-0.038433,-0.008507,-0.026423,-0.003203,-0.014813,-0.017215,-0.029225,-0.065857,0.039534,-0.031427,0.040034,-0.069360,0.084573,0.045940,0.025122,0.046640,0.007206,-0.061453,-0.023620,-0.073563,-0.024021,-0.061953,0.015413,0.020418,-0.026123,-0.003403,-0.000200,-0.049142,0.075565,0.080669,0.067458,...,0.012811,-0.001501,-0.005004,-0.059551,0.007907,-0.049643,-0.007707,0.061653,-0.032728,-0.055548,0.099386,0.009208,-0.022119,0.033429,-0.027023,-0.014613,0.014913,0.059952,0.058250,-0.017915,-0.022119,-0.085173,0.009108,0.025522,0.051144,-0.035330,-0.027824,-0.042036,-0.007907,0.017915,0.003603,-0.009508,0.023420,0.000400,-0.080669,0.065957,0.024721,0.006806,-0.016814,0.002903
963307,-0.013012,-0.015615,0.001001,0.014013,0.091786,-0.012312,0.062558,-0.042840,0.012512,0.009509,-0.024423,0.022921,0.024022,0.015915,-0.011811,-0.089484,-0.020119,0.028126,0.055852,0.040238,-0.072768,0.016315,0.016315,0.022321,0.016616,0.059155,-0.051949,-0.111504,-0.089684,0.003403,0.003503,0.006706,-0.082777,-0.043240,-0.003804,0.020519,-0.078073,-0.031930,-0.011811,-0.031029,...,-0.032630,-0.004104,0.031229,-0.026225,0.027225,0.018717,0.049947,0.030328,0.005605,0.017116,-0.019018,0.035533,-0.023722,-0.064260,0.017216,0.049847,0.055852,-0.006

In [71]:
df_joined = df_title.join(df_images, lsuffix="i", rsuffix="t")
df_joined.dropna(inplace=True)
df_joined.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_joined)
df_joined.describe()

,0i,1i,2i,3i,4i,5i,6i,7i,8i,9i,10i,11i,12i,13i,14i,15i,16i,17i,18i,19i,20i,21i,22i,23i,24i,25i,26i,27i,28i,29i,30i,31i,32i,33i,34i,35i,36i,37i,38i,39i,...,472t,473t,474t,475t,476t,477t,478t,479t,480t,481t,482t,483t,484t,485t,486t,487t,488t,489t,490t,491t,492t,493t,494t,495t,496t,497t,498t,499t,500t,501t,502t,503t,504t,505t,506t,507t,508t,509t,510t,511t
count,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5.140000e+03,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,...,5140.000000,5140.000000,5140.000000,5.140000e+03,5140.000000,5140.000000,5.140000e+03,5140.000000,5140.000000,5140.000000,5140.000000,5.140000e+03,5.140000e+03,5.140000e+03,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5.140000e+03,5140.000000,5140.000000,5140.000000,5140.000000,5.140000e+03,5140.000000,5140.000000,5140.000000,5.140000e+03,5.140000e+03,5140.000000,5.140000e+03,5140.000000,5140.000000,5140.000000,5140.000000,5.140000e+03
mean,-0.002737,0.001062,-0.001219,-0.009149,0.012182,0.010252,0.000007,-0.003772,-0.008084,-0.003586,-0.000480,0.010444,0.003460,0.004826,-0.000056,-0.009027,0.007581,0.000476,-0.001667,0.002012,-0.000495,0.008826,0.009485,0.001774,0.014156,-7.106920e-07,-0.007100,-0.004734,-0.016059,-0.005168,-0.005152,0.013548,-0.007314,-0.014442,0.003734,0.005778,-0.008203,-0.000762,-0.006366,-0.018776,...,0.000006,-0.000003,0.000004,6.887921e-07,0.000004,0.000008,4.505620e-07,-0.000011,-0.000005,-0.000008,0.000001,9.393994e-07,6.316692e-07,-3.171863e-07,0.000004,0.000002,-0.000001,-0.000004,0.000002,0.000001,-0.000001,-0.000009,6.304746e-07,0.000006,0.000002,-0.000005,-0.000007,3.173408e-07,0.000004,0.000005,0.000003,-9.058683e-07,-1.177182e-07,0.000005,3.472489e-07,0.000002,0.000007,-0.000002,-0.000006,6.476371e-07
std,0.026697,0.027851,0.033896,0.024909,0.041101,0.028328,0.032089,0.031879,0.029486,0.021623,0.029463,0.029739,0.031426,0.028049,0.028889,0.027399,0.031112,0.029415,0.032821,0.027107,0.030108,0.022766,0.032092,0.022415,0.027188,2.936284e-02,0.027245,0.027783,0.030128,0.028226,0.023838,0.024232,0.026326,0.026035,0.030201,0.026804,0.031042,0.028599,0.033008,0.032918,...,0.010433,0.010371,0.010397,1.037157e-02,0.010355,0.010326,1.032824e-02,0.010301,0.010258,0.010252,0.010220,1.019231e-02,1.017334e-02,1.015796e-02,0.010122,0.010139,0.010121,0.010053,0.010087,0.010030,0.010049,0.010030,9.953076e-03,0.009906,0.009949,0.009952,0.009891,9.851240e-03,0.009806,0.009799,0.009759,9.790527e-03,9.720373e-03,0.009680,9.689679e-03,0.009653,0.009682,0.009606,0.009660,9.619271e-03
min,-0.090519,-0.078698,-0.112035,-0.091643,-0.110474,-0.084650,-0.095333,-0.103400,-0.097243,-0.094766,-0.092080,-0.093137,-0.097315,-0.103895,-0.085708,-0.079762,-0.084441,-0.133902,-0.105805,-0.086913,-0.088613,-0.081531,-0.118406,-0.070987,-0.098302,-9.328294e-02,-0.092571,-0.083229,-0.106378,-0.089316,-0.085278,-0.119327,-0.092573,-0.102196,-0.089108,-0.084575,-0.088468,-0.083510,-0.103831,-0.101915,...,-0.041932,-0.041083,-0.037405,-4.200167e-02,-0.044545,-0.035072,-4.058682e-02,-0.038678,-0.036841,-0.033942,-0.037688,-3.889053e-02,-4.440624e-02,-3.585187e-02,-0.039810,-0.043064,-0.039882,-0.039672,-0.041791,-0.038325,-0.041437,-0.043206,-3.747640e-02,-0.038537,-0.038821,-0.042356,-0.038537,-4.157804e-02,-0.036134,-0.040799,-0.043415,-3.450660e-02,-4.037689e-02,-0.037619,-4.185891e-02,-0.042709,-0.035707,-0.050982,-0.038609,-3.662692e-02
25%,-0.021869,-0.018756,-0.026328,-0.027177,-0.021728,-0.009006,-0.022949,-0.027107,-0.029796,-0.017269,-0.021340,-0.010776,-0.018985,-0.013394,-0.020100,-0.026540,-0.014791,-0.018826,-0.023143,-0.017641,-0.022082,-0

In [44]:
def runUmapOnDf(df):
  umap_op = umap.UMAP(n_components=2, n_neighbors=10,min_dist=0.5)
  data_umap = umap_op.fit_transform(df.values)
  data_umap_df = pd.DataFrame(data=data_umap, columns=['x', 'y'])
  data_umap_df['id'] = df.index
  return data_umap_df.set_index('id')

In [72]:
df_joined_umap = runUmapOnDf(df_joined).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [60]:
df_joined_umap3 = runUmapOnDf(df_images).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap3).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [61]:
df_joined_umap2 = runUmapOnDf(df_title).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap2).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [49]:
df_joined_umap2

,x,y,sammlung
id,,,
1486397,3.327157,6.909046,mek
1570538,3.721031,12.012905,mek
525580,6.183940,10.559856,mek
1442848,-0.287874,10.157651,mek
962622,0.262986,15.192502,ang
...,...,...,...
984816,5.278814,9.231367,mek
1479834,1.868817,5.283248,mek
961858,-6.149833,11.688212,ang


In [57]:
df_joined_umap.to_csv("pca-use-2d.csv")